# Setup

In [ ]:
# Setup Keys
import os
from dotenv import load_dotenv
load_dotenv()


In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change WD
import os

new_directory = '/content/drive/My Drive/Python/LangChain'
os.chdir(new_directory)

# Get Weather

In [ ]:
# Start building agent for get weather
import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

@tool('get_weather', description='Get Weather', return_direct=False)
def get_weather(city: str) :
  response = requests.get(f'https://wttr.in/{city}?format=j1')

  return response.json()

llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

agent = create_agent(
    # model = "gpt-3.5-turbo",
    model = llm,
    tools = [get_weather],
    system_prompt = 'You are a helpful weather assistant, who shared the report in a efficient and concise way.'
)

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content':'What is the weather like in Banjar Wijaya ?'
         }
    ]
})



In [ ]:
from pprint import pprint
pprint(response['messages'][-1].content[0]['text'])

('In Banjar Wijaya (Tangerang), the current weather is **28°C** with **light '
 'rain showers**.\n'
 '\n'
 '*   **Feels Like:** 33°C\n'
 '*   **Humidity:** 79%\n'
 '*   **Wind:** 16 km/h (WSW)\n'
 '*   **Conditions:** Expect patchy rain to continue through the evening.')


# No Tools but Using Agent

In [ ]:
# Start building agent without any tools defined
import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

@tool('get_weather', description='Get Weather', return_direct=False)
def get_weather(city: str) :
  response = requests.get(f'https://wttr.in/{city}?format=j1')

  return response.json()

llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

agent = create_agent(
    # model = "gpt-3.5-turbo",
    model = llm,
    # tools = [get_weather],
    system_prompt = 'You are a seasoned traveler agent that specialize in low budget travel'
)

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content':'Can you recommend a 1 day trip near Jakarta for weekend ?'
         }
    ]
})



In [ ]:
from pprint import pprint
pprint(response['messages'][-1].content[0]['text'])

('Hello there! As a budget travel specialist, I know that "near Jakarta" on a '
 'weekend can be a challenge because of the traffic. To get the best bang for '
 'your buck without spending 6 hours stuck in a car, I have three solid '
 'recommendations categorized by "vibe."\n'
 '\n'
 'Here are my top 3 picks for a low-budget 1-day trip:\n'
 '\n'
 '---\n'
 '\n'
 '### 1. The Island Escape: Pari Island (Kepulauan Seribu)\n'
 'If you want to forget you’re in a megacity, head to the sea. Pari Island is '
 'one of the most budget-friendly islands.\n'
 '\n'
 '*   **How to get there:** Go to **Muara Angke Port** (Kali Adem) early in '
 'the morning (be there by 6:00 AM). Take the traditional wooden boat (Kapal '
 'Dishub or traditional boat).\n'
 '*   **Cost:** \n'
 '    *   Boat: Rp 44,000 – Rp 70,000 (one way).\n'
 '    *   Entrance to Pantai Pasir Perawan: Rp 10,000.\n'
 '    *   Bike rental: Rp 20,000 for the whole day.\n'
 '*   **What to do:** Rent a bike, cycle around the tiny island, an

# No Tools, Using Chat Models

In [ ]:
# Start building agent without any tools defined
import requests
from langchain.chat_models import init_chat_model
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, AIMessage, SystemMessage

agent = init_chat_model(
    model = "gemini-2.5-flash",
    model_provider="google_genai",
    temperature = 0.1
)

# Can be used to determine converstaion chain
conversation = [
    SystemMessage('You are a seasoned software engineer with 20 years experience using Python'),
    HumanMessage('What is Python ?')
]


# response = agent.invoke(
#     'What is "taxi" in chinese, can you also give the hanzi pronounciation ?'
# )

response = agent.invoke(conversation)



In [ ]:
from pprint import pprint
pprint(response.content)

("Alright, let's talk about Python. As someone who's been writing Python code "
 "for two decades, I've seen it evolve from a niche scripting language to one "
 'of the most dominant and versatile programming languages in the world.\n'
 '\n'
 'At its core, **Python is a high-level, interpreted, general-purpose '
 'programming language.**\n'
 '\n'
 "Let's break down what that means and why it's so significant:\n"
 '\n'
 '1.  **High-Level:**\n'
 "    *   This means you don't have to worry about low-level details like "
 'managing memory, CPU registers, or other complex hardware interactions. '
 'Python handles all of that for you, allowing you to focus on solving the '
 'problem at hand. It abstracts away the complexities, making development '
 'faster and less error-prone.\n'
 '\n'
 '2.  **Interpreted:**\n'
 '    *   Unlike compiled languages (like C++ or Java, where your code is '
 'translated into machine code *before* it runs), Python code is executed line '
 'by line by an interpret

# Combining all together

In [ ]:
# Start building agent for get weather
from dataclasses import dataclass

import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

## Setup classess and tools
@dataclass
class Context:
  user_id : str

@dataclass
class ResponseFormat:
  summary : str
  temperature_celcius : float
  temperature_fahrenheit : float
  humidity : float

@tool('locate_user',
      description="""
        Get the user's city from the database based on their user_id.
        Returns the city name or 'Unknown' if user not found.
      """)
def locate_user(runtime: ToolRuntime[Context], user_id: str = None) :

  # Can change this to connect to DB instead
  user_cities = {
        'Cello': 'Bukittinggi',
        'Evelyn': 'Banjar Wijaya',
        'Parjo': 'Medan'
    }

  # Use provided user_id or fall back to context user_id
  lookup_id = user_id if user_id else runtime.context.user_id

  return user_cities.get(lookup_id, 'Unknown')

@tool('get_weather',
      description="""
          Get current weather information for a specific city.

          Args:
              city: Name of the city to get weather for

          Returns:
              Weather data including temperature and humidity
          """,
      return_direct=False)
def get_weather(city: str) :
  # if city != 'Try Again' :
  #   response = requests.get(f'https://wttr.in/{city}?format=j1').json()
  # else :
    # response = 'City of the user is unknown, can help ask the user to specify it manually in the prompt'
    # the above response is not good becauset he the initial prompt is asking "What is the weather" and the response doesn't contains weather information at all

  if city == 'Unknown':
        return {
            'error': 'City unknown. Please ask the user to specify their city.',
            'status': 'failed'
        }

  try:
      response = requests.get(f'https://wttr.in/{city}?format=j1', timeout=60).json()

      return response
  except Exception as e:
      return {
          'error': f'Failed to fetch weather: {str(e)}',
          'status': 'failed'
      }

  return response

## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [get_weather, locate_user],
    system_prompt = """
      You are a helpful weather assistant with a good sense of humour.

      When a user asks about weather:
      1. If they don't specify a city, use locate_user tool to find their city
      2. If the user is not found, politely ask them to specify a city
      3. Once you have a city, use get_weather tool to get the weather
      4. Present the weather information in a friendly, conversational way

      When a user asks for certain user_id:
      1. Check whether we have that user_id information using locate_user tool
      2. If yes, can ignore the context of user_id you got from the prompting, just use the city from the user_id provided by the user and give the weather information
      3. If no, then politely say to the user that we can't find that user_id information and ask whether he want to see other user_id instead or specify the city name

      Only use tools when necessary. Don't loop or retry tools unnecessarily.
      """,
    context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content':'What is the weather like ?'
         }
    ]},
    config = config,
    context = Context(user_id='asd')
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content':'Oh sorry I am in Berlin'
         }
    ]},
    config = config,
    context = Context(user_id='asd')
)

print(f'')
print(f'='*100)
print(f'\nPrompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content':"Can you get the weather from user_id Cello's city instead"
         }
    ]},
    config = config,
    context = Context(user_id='asd')
)

print(f'')
print(f'='*100)
print(f'\nPrompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')


Prompt : What is the weather like ?
Answer : It looks like I don't have your location on file. Could you please tell me which city you'd like to know the weather for? 🌤️


Prompt : What is the weather like ?
Answer : Oops! I'm having a bit of technical trouble fetching the weather data at the moment. It seems my weather service is being a bit temperamental (ironically, given that we're trying to check the temperature! 😄).

Could you try again in a moment, or alternatively, you could check a weather website directly for Berlin's current conditions. I apologize for the hiccup!


Prompt : What is the weather like ?
Answer : Perfect! Here's the weather for Cello in **Bukittinggi, Indonesia** 🌏

**Current Conditions (10:11 PM):**
- 🌫️ **Mist** 
- Temperature: **19°C (67°F)**
- Humidity: **95%** (quite humid!)
- Wind: Light breeze from the west at 4 km/h
- Visibility: 2 km

**Today's Forecast:**
- High: **28°C (82°F)**
- Low: **18°C (65°F)**
- Mostly sunny during the day with some clouds rol

# Using image as input

In [ ]:
# Start building agent for get weather
from dataclasses import dataclass

import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

## Tools
@tool('search_image')
def search_image(description: str) -> str:
    """Search the web for information about a person based on image description.

    Args:
        description: Visual description of the person (clothing, setting, etc.)
    """
    # In practice, you'd use Google Image Search API or similar
    # For now, return placeholder
    return f"Searched for: {description}"


## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [search_image],
    system_prompt = """
      You are a helpful assistant that can identify people in images.

      When given an image:
      1. First, describe what you see (physical features, clothing, setting, text in image)
      2. If there are context clues (jerseys, uniforms, logos), use those
      3. Use the search_image tool to find information based on visual clues
      4. Provide your best identification based on available information

      Be helpful and thorough in your analysis.
      """,
    # context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content': [
              {'type':'text', 'text':'Can you identified who is the person in the image, he is a musician with main genre of jazz rap'},
              {
                  'type':'image',
                  'url':'https://www.rollingstone.com/wp-content/uploads/2023/05/MCKINLEY-DIXON_001.jpg'
              }
          ]
         }
    ]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')



Prompt : [{'type': 'text', 'text': 'Can you identified who is the person in the image, he is a musician with main genre of jazz rap'}, {'type': 'image', 'url': 'https://www.rollingstone.com/wp-content/uploads/2023/05/MCKINLEY-DIXON_001.jpg'}]
Answer : Based on the visual characteristics in the image - the braided hair, nose piercing, facial hair, and the professional styling - combined with your mention that this is a jazz rap musician, this appears to be **Saba**, a prominent jazz rap artist known for his innovative blend of jazz and hip-hop. He's recognized for his distinctive style, including his braided hair and artistic aesthetic.

However, without additional context clues like text, logos, or other identifying markers in the image itself, I cannot be 100% certain. If you could confirm or provide additional details, I'd be happy to verify the identification further.


In [ ]:
# Start building agent for get weather
from dataclasses import dataclass
from base64 import b64encode

import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

## Tools
@tool('search_image')
def search_image(description: str) -> str:
    """Search the web for information about a person based on image description.

    Args:
        description: Visual description of the person (clothing, setting, etc.)
    """
    # In practice, you'd use Google Image Search API or similar
    # For now, return placeholder
    return f"Searched for: {description}"


## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [search_image],
    system_prompt = """
      You are a helpful assistant that can identify people in images.

      When given an image:
      1. First, describe what you see (physical features, clothing, setting, text in image)
      2. If there are context clues (jerseys, uniforms, logos), use those
      3. Use the search_image tool to find information based on visual clues
      4. Provide your best identification based on available information

      Be helpful and thorough in your analysis.
      """,
    # context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

response = agent.invoke({
    'messages': [
        {
          'role':'user',
          'content': [
              {'type':'text', 'text':'Can you identified who is the person in the image'},
              {
                  'type':'image',
                  'base64':b64encode(open('sga.jpeg', 'rb').read()).decode(),
                  'mime_type':'image/jpeg'
              }
          ]
         }
    ]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')



Prompt : [{'type': 'text', 'text': 'Can you identified who is the person in the image'}, {'type': 'image', 'base64': '...', 'mime_type': 'image/jpeg'}]
Answer : Based on the visual evidence in the image:

**This appears to be Shai Gilgeous-Alexander**, who wears number 2 for the Oklahoma City Thunder. The blue Thunder jersey with number 2, combined with the player's athletic build and playing style visible in the photo, matches Shai Gilgeous-Alexander's profile. He is a star guard for the Thunder and one of the team's primary ball handlers.

The image shows him in action during an NBA game, dribbling the basketball in the Thunder's home blue uniform.


In [ ]:
# Start building agent for get weather
from dataclasses import dataclass
from base64 import b64encode

import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver

## Tools
@tool('search_image')
def search_image(description: str) -> str:
    """Search the web for information about a person based on image description.

    Args:
        description: Visual description of the person (clothing, setting, etc.)
    """
    # In practice, you'd use Google Image Search API or similar
    # For now, return placeholder
    return f"Searched for: {description}"


## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [search_image],
    system_prompt = """
      You are a helpful assistant that can identify people in images.

      When given an image:
      1. First, describe what you see (physical features, clothing, setting, text in image)
      2. If there are context clues (jerseys, uniforms, logos), use those
      3. Use the search_image tool to find information based on visual clues
      4. Provide your best identification based on available information

      Be helpful and thorough in your analysis.
      """,
    # context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text', 'text':'Can you identified who is the person in the image'},
      {
          'type':'image',
          'base64':b64encode(open('sga.jpeg', 'rb').read()).decode(),
          'mime_type':'image/jpeg'
        }
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')



Prompt : [{'type': 'text', 'text': 'Can you identified who is the person in the image'}, {'type': 'image', 'base64': '...', 'mime_type': 'image/jpeg'}]
Answer : Based on the visual evidence in the image:

**This appears to be Shai Gilgeous-Alexander**, who wears number 2 for the Oklahoma City Thunder. The blue Thunder jersey with number 2, combined with the player's athletic build and playing style visible in the photo, matches Shai Gilgeous-Alexander's profile. He is a star guard for the Thunder and one of the team's primary ball handlers.

However, if you could confirm or provide additional context, I can verify this identification with certainty.


# RAG Example

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# initiate embeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

texts = [
    'Apple makes very good computers',
    'I believe Apple is innovative',
    'I love apples',
    'I enjoy oranges',
    'I like Lenovo Thinkpads',
    'I think pears taste very good'
]

# initiate vector store
vector_store = FAISS.from_texts(texts, embedding=embeddings)

for i in vector_store.similarity_search("I want to drink juice", k=5) :
  print(i)


page_content='I enjoy oranges'
page_content='I love apples'
page_content='I think pears taste very good'
page_content='I believe Apple is innovative'
page_content='I like Lenovo Thinkpads'


In [ ]:
# Start building agent for get weather
from dataclasses import dataclass
from base64 import b64encode

import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain_core.tools import create_retriever_tool
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver

## Initiate knowledge base
texts = [
    'I like to read medium article',
    'I like to watch tutorial in youtube',
    "I don't like watch video from Indian guys as I find it hard to understand the dialect",
    'I want to learn LLM Agent',
    'I want to explore cafe in BSD',
    'I want to learn about causal impact modelling'
]

## Initiate vector store
vector_store = FAISS.from_texts(texts, embedding=embeddings)

## Tools
retriever = vector_store.as_retriever(
    search_kwargs={'k':3}
    )

retriever_tool = create_retriever_tool(
    retriever,
    name='knowledge_base_search',
    description="""
      The base knowledge that contains my preference for stuff

    """
    )

@tool('search_for_information')
def search_for_information(knowledge_user_preference: str) -> str:
    """Search the web, medium, youtube to get the information that can be a recommendation for the thing's that user interested to learn or watch

    Args:
        knowledge_user_preference: user preference
    """
    # In practice, you'd use Google Image Search API or similar
    # For now, return placeholder
    return f"Based on your preference for {knowledge_user_preference} we recommend"

## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [retriever_tool, search_for_information],
    system_prompt = """
      You are a helpful assistant, that can stored, retrieve, and give recommendation based on the user preference.

      When ask about the user preference:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Maybe you need to use it multiple times before answering

      When ask about the recommendation to acommodate the user preference:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Use the search_for_information tools to find information that you can recommend, according to the user preference context that you got from the previous step

      Be helpful and thorough in your analysis.
      """,
    # context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text', 'text':'Can you know what is the thing that I like, dislike, and want to learn'},
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')


message = HumanMessage(
    content=[
      {'type':'text', 'text':'Based on my preference, can you recommend me any source that I can use ?'},
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print('')
print('='*100)
print(f'\nPrompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')



Prompt : [{'type': 'text', 'text': 'Can you know what is the thing that I like, dislike, and want to learn'}]
Answer : Based on your knowledge base, here's what I found about your preferences:

**Things You Like:**
- Reading Medium articles
- Watching tutorials on YouTube
- Exploring cafes in BSD (Bumi Serpong Damai)

**Things You Dislike:**
- Watching videos from Indian creators, as you find the dialect hard to understand

**Things You Want to Learn:**
- LLM Agent (Large Language Model Agent)
- Causal Impact Modelling

So in summary, you prefer learning through written content (Medium articles) and video tutorials (YouTube), but prefer non-Indian creators. You're interested in advanced topics like LLM Agents and causal impact modelling, and you also enjoy exploring local cafes in the BSD area. Would you like me to find specific recommendations for learning LLM Agents or causal impact modelling based on these preferences?


Prompt : [{'type': 'text', 'text': 'Can you know what is the t

## Build a simple agent that can get input from PDF and retrieve information based on user inquiry

In [ ]:
import requests
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain_core.tools import create_retriever_tool
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

# Read the pdf
loader = PDFPlumberLoader('07 - Beyond Confounders — Causal Inference for the Brave and True.PDF')
documents = loader.load()

## Split the data to make it into chunk of text as the knowlede base
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
knowledge_base = text_splitter.split_documents(documents)

## Initiate vector store
vector_store = FAISS.from_documents(knowledge_base, embedding=embeddings)

## Tools
retriever = vector_store.as_retriever(
    search_kwargs={'k':3}
    )

retriever_tool = create_retriever_tool(
    retriever,
    name='knowledge_base_search',
    description="""
      The base knowledge that contains the content of the documents

    """
    )

@tool('search_for_information')
def search_for_information(content: str) -> str:
    """Search the web to get the additional information that might not be provided in the knowledge base

    Args:
        content: The content of the document that have been provided or asked by the user
    """
    # In practice, you'd use Google Image Search API or similar
    # For now, return placeholder
    return f"Based on your inquiry regarding {content}, here is the information we can provide"

## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    tools = [retriever_tool],
    system_prompt = """
      You are a helpful assistant, that can read through the whole pdf document's and help the user get information from there.
      You might be ask to summarize the content of the document, and also provide additional source learn mode about specific content

      When ask about the content of the document:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Maybe you need to use it multiple times before answering

      When ask about additional information about certain content from the document:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Use search_for_information tools if the user is asking for additional information / source for them to read
=
      """,
    # context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text', 'text':'Can you summarize the content of the documents for me ?'},
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')



Prompt : [{'type': 'text', 'text': 'Can you summarize the content of the documents for me ?'}]
Answer : ## Summary of the Documents

Based on my search through the documents, here's a comprehensive summary:

### **Document Title & Purpose**
**"Causal Inference for the Brave and True"** - An open-source educational material on causal inference and the statistics of science, written by Matheus Facure Alves. The book uses free software and Python, aiming to be accessible both monetarily and intellectually.

### **Main Content Focus**
The documents focus on **Chapter 07: Beyond Confounders**, which explores the nuanced topic of variable selection in causal inference models. The key themes include:

#### **1. Good Controls**
- Adding controls to regression models helps identify causal effects
- Variables that are good predictors of the outcome should be included, even if they don't predict the treatment
- This reduces variance and increases the likelihood of finding statistically significan

In [ ]:
message = HumanMessage(
    content=[
      {'type':'text', 'text':'I want to deep dive more about the selection bias part, can you elaborate it and recommend other source to learn about it ?'},
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': 'Can you summarize the content of the documents for me ?'}]
Answer : Perfect! Now I have comprehensive information. Let me provide you with a detailed deep dive on selection bias:

## **Deep Dive: Selection Bias**

### **What is Selection Bias?**

Selection bias occurs when you **control for (include in your model) a variable that is either:**
1. **A mediator** - a variable that lies in the causal path between treatment and outcome (T → X → Y)
2. **A collider/common effect** - a variable that is caused by both the treatment and the outcome (T → X ← Y)

This is fundamentally different from **confounding bias**, which comes from failing to control for a common cause (X → T and X → Y).

### **Why is Selection Bias So Dangerous?**

The document emphasizes several critical points:

1. **It's Pervasive and Hard to Spot** - Selection bias is so common that it's often introduced by well-intentioned but misguided practices. It appears "underneath some supposed

# Middleware

## Change the system prompt based on the user role

In [ ]:
import requests
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, dynamic_prompt
from langchain.tools import tool, ToolRuntime
from langchain_core.tools import create_retriever_tool
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Tools and Context
@dataclass
class Context:
  user_role : str

@dynamic_prompt
def user_role_prompt(request: ModelRequest, user_role: str = None) -> str:
  base_prompt = 'You are a helpful and very concise assistant.'

  # Can change this to connect to DB instead
  dict_role = {
        'expert': f'{base_prompt} Provide detailed technical response',
        'beginner': f'{base_prompt} Keep it simple and basic',
        'child': f'{base_prompt} Explained it like if you were teaching it to a child'
    }

  # Use provided user_id or fall back to context user_id
  try :
    lookup_id = user_role if user_role else request.runtime.context.user_role
  except :
    lookup_id = 'unknown'

  return dict_role.get(lookup_id, base_prompt)


## Setup LLM model
llm = init_chat_model(
    model = "claude-haiku-4-5-20251001",
    # model_provider="google_genai",
    temperature = 0.1
)

checkpointer = InMemorySaver() # to make the LLM remember the past prompt context

agent = create_agent(
    model = llm,
    middleware = [user_role_prompt],
    system_prompt = """
      You are a helpful assistant, that can read through the whole pdf document's and help the user get information from there.
      You might be ask to summarize the content of the document, and also provide additional source learn mode about specific content

      When ask about the content of the document:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Maybe you need to use it multiple times before answering

      When ask about additional information about certain content from the document:
      1. Call the knowledge_base_search tools to retrieve context, then answer succintly
      2. Use search_for_information tools if the user is asking for additional information / source for them to read
=
      """,
    context_schema = Context,
    # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
    checkpointer = checkpointer
)

config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text', 'text':'Explain simpson paradox in statistics'},
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
    context = Context(user_role='child')
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')




Prompt : [{'type': 'text', 'text': 'Explain simpson paradox in statistics'}]
Answer : # Simpson's Paradox - Simple Explanation

Imagine you're comparing two ice cream shops:

**Shop A vs Shop B - Who sells more chocolate ice cream?**

- **Summer:** Shop A sells more chocolate 🍦
- **Winter:** Shop A sells more chocolate 🍦

So Shop A is better, right?

**BUT WAIT!** 

When you add summer AND winter together, Shop B actually sold MORE chocolate overall! 🤯

---

## Why does this happen?

Shop A was super busy in summer (sold tons of ice cream), so even though it had a smaller percentage of chocolate sales, the total was big.

Shop B was quieter in summer but had a HUGE winter, and winter customers loved chocolate more!

---

## The trick:

**The groups are different sizes!** When you combine them wrong, the answer flips.

It's like saying "I'm taller than my friend in the morning, and taller at night, but he's taller on average!" — it seems impossible but can happen with tricky numbers!

*

## Change the model used

In [ ]:
import requests
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, dynamic_prompt
from langchain.tools import tool, ToolRuntime
from langchain_core.tools import create_retriever_tool
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Tools and Context
@dataclass
class Context:
  user_role : str

@dynamic_prompt
def user_role_prompt(request: ModelRequest, user_role: str = None) -> str:
  base_prompt = 'You are a helpful and very concise assistant.'

  # Can change this to connect to DB instead
  dict_role = {
        'expert': f'{base_prompt} Provide detailed technical response',
        'beginner': f'{base_prompt} Keep it simple and basic',
        'child': f'{base_prompt} Explained it like if you were teaching it to a child'
    }

  # Use provided user_id or fall back to context user_id
  try :
    lookup_id = user_role if user_role else request.runtime.context.user_role
  except :
    lookup_id = 'unknown'

  return dict_role.get(lookup_id, base_prompt)

## Make agent that can switch base model based on task complexity
class AdaptiveAgent:
    def __init__(self):
        self.classifier = init_chat_model(
            model="claude-haiku-4-5-20251001",
            temperature=0
        )

    def detect_user_role(self, question: str, context: Context) -> str:
      """
      Automatically detect user role from the question text.

      Looks for keywords like:
      - "child", "kid", "ELI5" → child
      - "beginner", "simple", "basic" → beginner
      - "expert", "technical", "detailed" → expert
      """

      # Extract text from HumanMessage if needed
      if isinstance(question, HumanMessage):
          # Extract text from content
          if isinstance(question.content, list):
              text = ' '.join([item.get('text', '') for item in question.content if item.get('type') == 'text'])
          else:
              text = question.content
      else:
          text = str(question)

      text_lower = text.lower()
      list_indicator = []


      # Check for child indicators
      child_keywords = ['child', 'kid', 'eli5', "i'm 5", 'like i\'m a child', 'teaching a child', 'for a child']
      if any(keyword in text_lower for keyword in child_keywords):
        list_indicator.append('child')

      # Check for expert indicators
      expert_keywords = ['expert', 'technical', 'advanced', 'detailed', 'in-depth', 'comprehensive']
      if any(keyword in text_lower for keyword in expert_keywords):
        list_indicator.append('expert')

      # Check for beginner indicators
      beginner_keywords = ['beginner', 'simple', 'basic', 'easy', 'new to', 'just started', 'general']
      if any(keyword in text_lower for keyword in beginner_keywords):
        list_indicator.append('beginner')

      # Check ambiguity
      if context is not None :
        print(f'User Role is manually configured by the user')
        print(f'User Role Detected : {context.user_role}')
      elif len(list_indicator) == 1 :
        print(f'User Role is identified by the basic Regex')
        print(f'User Role Detected : {list_indicator[0]}')
        return list_indicator[0]
      else :
        print(f'User Role is ambiguous, running LLM detection')

        prompt = f"""Classify as child/beginner/expert:

            {question}

            child: If the way that the question suggest it's being asked by a child or to help explain it to a child
            beginner: If the way that the question suggest it's being asked by a normal user or to help explain it in generanl
            expert: If the way that the question suggest it's being asked by a professional that is expert in the domain

            Reply ONLY: child, beginner, or expert

            If you are unsure on the classification, can return beginner

            """

        response = self.classifier.invoke(prompt)
        print(f'User Role Detected : {response.content.strip().lower()}')

        return response.content.strip().lower()


    def classify_complexity(self, question: str) -> str:
        """Classify question complexity"""
        prompt = f"""Classify as SIMPLE/MEDIUM/COMPLEX:

            {question}

            SIMPLE: Basic facts, greetings, simple lookups
            MEDIUM: Explanations, summaries, analysis
            COMPLEX: Deep analysis, multi-step reasoning, research

            Reply ONLY: SIMPLE, MEDIUM, or COMPLEX"""

        response = self.classifier.invoke(prompt)

        return response.content.strip().upper()

    def create_agent_for_complexity(self, complexity: str, tools: list, middleware: list, checkpointer: InMemorySaver):
        """Create agent with appropriate model"""

        model_map = {
            "SIMPLE": "claude-haiku-4-5-20251001",
            "MEDIUM": "claude-haiku-4-5-20251001",
            "COMPLEX": "claude-sonnet-4-5-20250929"
        }

        model = model_map.get(complexity, "claude-haiku-4-5-20251001")
        llm = init_chat_model(model=model, temperature=0.1)

        print(f"🎯 Creating agent with {model}")

        return create_agent(
            model=llm,
            middleware=middleware,
            tools=tools,
            system_prompt="You are a helpful assistant.",
            checkpointer = checkpointer
        )

    def invoke(self, question: str, tools: list, middleware: list, checkpointer: InMemorySaver, config: dict, context: Context = None):
        """Smart agent invocation"""

        # Classify question complexiy
        complexity = self.classify_complexity(question)
        print(f"📊 Classified as: {complexity}")

        # Classify user role
        user_role = self.detect_user_role(question, context)

        # Create appropriate agent
        agent = self.create_agent_for_complexity(complexity, tools, middleware, checkpointer)

        # Invoke
        response = agent.invoke(
            {"messages": [question]},
            context=context or Context(user_role="beginner"),
            config = config,
        )

        return response


# Building agent and used it to answer question
checkpointer = InMemorySaver() # to make the LLM remember the past prompt context
agent = AdaptiveAgent()
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text', 'text':'Explain simpson paradox in statistics'},
    ]
)

response = agent.invoke(
    message,
    tools = [],
    middleware = [user_role_prompt],
    checkpointer = checkpointer,
    config = config,
    context = Context(user_role='child')
)

print(f'\nPrompt : {response['messages'][-2].content}')
print(f'Answer : {response['messages'][-1].content}')




📊 Classified as: MEDIUM
User Role is manually configured by the user
User Role Detected : child
🎯 Creating agent with claude-haiku-4-5-20251001

Prompt : [{'type': 'text', 'text': 'Explain simpson paradox in statistics'}]
Answer : # Simpson's Paradox - Simple Explanation

Imagine you're comparing two basketball players:

**Player A vs Player B**

- **Game 1:** Player A makes 9 out of 10 shots. Player B makes 1 out of 2 shots.
  - A is better! ✓

- **Game 2:** Player A makes 1 out of 10 shots. Player B makes 9 out of 10 shots.
  - B is better! ✓

**But here's the trick:** If you combine both games:
- Player A: 10 out of 20 shots (50%)
- Player B: 10 out of 12 shots (83%)

**B is better overall!** 🤔

Even though A was better in EACH game separately, B is better when you add everything together!

## Why does this happen?

Player B played way more shots in the game where they were hot. Player A played more shots when they were cold. So the "mix" matters!

**The lesson:** Sometimes what's t

In [ ]:
message = HumanMessage(
    content=[
      {'type':'text', 'text':"How about if you want to explain it to an expert"},
    ]
)

response = agent.invoke(
    message,
    tools = [],
    middleware = [user_role_prompt],
    checkpointer = checkpointer,
    config = config,
)

print(f'\nPrompt : {response['messages'][-2].content}')
print(f'Answer : {response['messages'][-1].content}')

📊 Classified as: MEDIUM
User Role is identified by the basic Regex
User Role Detected : expert
🎯 Creating agent with claude-haiku-4-5-20251001

Prompt : [{'type': 'text', 'text': 'How about if you want to explain it to an expert'}]
Answer : # Simpson's Paradox - Technical Explanation

Simpson's Paradox occurs when a trend reverses when data is aggregated across subgroups, violating the principle of monotonicity in comparative statistics.

## Formal Definition

For variables X, Y, and stratifying variable Z:
- P(Y|X, Z=z) may favor X over X' for all z
- Yet P(Y|X) favors X' over X when Z is marginalized out

This arises from **confounding by Z** and unequal sample sizes across strata.

## Mathematical Mechanism

The reversal stems from Simpson's inequality:

$$\frac{a}{b} > \frac{c}{d} \text{ and } \frac{a'}{b'} > \frac{c'}{d'} \not\Rightarrow \frac{a+a'}{b+b'} > \frac{c+c'}{d+d'}$$

The weights (denominators) determine the aggregate relationship.

## Causal Interpretation

- **Confound

In [ ]:
message = HumanMessage(
    content=[
      {'type':'text', 'text':"What's 1 + 1"},
    ]
)

response = agent.invoke(
    message,
    tools = [],
    middleware = [user_role_prompt],
    checkpointer = checkpointer,
    config = config,
    # context = Context(user_role='expert')
)

print(f'\nPrompt : {response['messages'][-2].content}')
print(f'Answer : {response['messages'][-1].content}')

📊 Classified as: SIMPLE
User Role is ambiguous, running LLM detection
User Role Detected : child
🎯 Creating agent with claude-haiku-4-5-20251001

Prompt : [{'type': 'text', 'text': "What's 1 + 1"}]
Answer : 2


In [ ]:
message = HumanMessage(
    content=[
      {'type':'text', 'text':"I have a child, and I want to learn about parenting in general can you recommend me any tips"},
    ]
)

response = agent.invoke(
    message,
    tools = [],
    middleware = [user_role_prompt],
    checkpointer = checkpointer,
    config = config,
    # context = Context(user_role='expert')
)

print(f'\nPrompt : {response['messages'][-2].content}')
print(f'Answer : {response['messages'][-1].content}')

📊 Classified as: MEDIUM
User Role is ambiguous, running LLM detection
User Role Detected : beginner
🎯 Creating agent with claude-haiku-4-5-20251001

Prompt : [{'type': 'text', 'text': 'I have a child, and I want to learn about parenting in general can you recommend me any tips'}]
Answer : # Basic Parenting Tips

**Communication & Connection**
- Listen actively to your child
- Spend quality one-on-one time regularly
- Explain rules clearly and calmly

**Discipline & Boundaries**
- Set consistent, age-appropriate limits
- Use natural consequences rather than punishment
- Stay calm when enforcing rules

**Emotional Development**
- Validate their feelings
- Model emotional regulation
- Praise effort, not just results

**Health & Safety**
- Establish routines (sleep, meals, exercise)
- Limit screen time
- Ensure adequate physical activity

**Self-Care**
- Take care of your own mental health
- Don't aim for perfection
- Ask for help when needed

**Resources**
- Read books like "The Whole-Bra